In [1]:
!pip3 install catboost
!pip3 install hyperopt==0.2.5

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached hyperopt-0.2.5-py2.py3-none-any.whl (965 kB)


In [2]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import hyperopt
from numpy.random import RandomState
from catboost.utils import select_threshold

In [3]:
csv_path = "../../data/trades/processed/trades-df-2021_11_1_0_0_0-2021_12_21_0_0_0.csv"
df = pd.read_csv(csv_path)

In [4]:
x_columns = list(filter(lambda column: not 'punch' in column, df.columns))
y_columns = list(filter(lambda column: 'punch' in column, df.columns))
print(x_columns)
print(y_columns)

['trade_volume-BUY-6-sec', 'moving_average_price-BUY-6-sec', 'weighted_moving_average_price-BUY-6-sec', 'exp_moving_average_price-BUY-6-sec', 'stochastic_oscillator-BUY-6-sec', 'trade_volume-BUY-12-sec', 'moving_average_price-BUY-12-sec', 'weighted_moving_average_price-BUY-12-sec', 'exp_moving_average_price-BUY-12-sec', 'stochastic_oscillator-BUY-12-sec', 'trade_volume-BUY-18-sec', 'moving_average_price-BUY-18-sec', 'weighted_moving_average_price-BUY-18-sec', 'exp_moving_average_price-BUY-18-sec', 'stochastic_oscillator-BUY-18-sec', 'trade_volume-BUY-24-sec', 'moving_average_price-BUY-24-sec', 'weighted_moving_average_price-BUY-24-sec', 'exp_moving_average_price-BUY-24-sec', 'stochastic_oscillator-BUY-24-sec', 'trade_volume-BUY-30-sec', 'moving_average_price-BUY-30-sec', 'weighted_moving_average_price-BUY-30-sec', 'exp_moving_average_price-BUY-30-sec', 'stochastic_oscillator-BUY-30-sec', 'trade_volume-BUY-36-sec', 'moving_average_price-BUY-36-sec', 'weighted_moving_average_price-BUY-36

In [5]:
X = df[x_columns]
secs = list(sorted(list(set(map(lambda x : int(''.join(x)), [[s for s in column if s.isdigit()] for column in x_columns])))))
delta_sec = secs[1] - secs[0]
max_sec = secs[-1]
min_sec = secs[0]

for column in x_columns:
    name, side, sec, _ = column.split('-')
    if not 'price' in name and int(sec) != min_sec:
        X[f'{name}-{side}-{sec}-sec'] -= X[f'{name}-{side}-{str(int(sec) - delta_sec)}-sec']
    elif 'price' in name:
        if int(sec) != max_sec:
            X[f'{name}-{side}-{sec}-sec'] /= X[f'{name}-{side}-{str(int(sec) + delta_sec)}-sec']
        else:
            X = X.drop(f'{name}-{side}-{sec}-sec', axis=1)

X

<ipython-input-5-bcd2e7a3d259>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[f'{name}-{side}-{sec}-sec'] /= X[f'{name}-{side}-{str(int(sec) + delta_sec)}-sec']
<ipython-input-5-bcd2e7a3d259>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[f'{name}-{side}-{sec}-sec'] -= X[f'{name}-{side}-{str(int(sec) - delta_sec)}-sec']


,trade_volume-BUY-6-sec,moving_average_price-BUY-6-sec,weighted_moving_average_price-BUY-6-sec,exp_moving_average_price-BUY-6-sec,stochastic_oscillator-BUY-6-sec,trade_volume-BUY-12-sec,moving_average_price-BUY-12-sec,weighted_moving_average_price-BUY-12-sec,exp_moving_average_price-BUY-12-sec,stochastic_oscillator-BUY-12-sec,...,weighted_moving_average_price-SELL-48-sec,exp_moving_average_price-SELL-48-sec,stochastic_oscillator-SELL-48-sec,trade_volume-SELL-54-sec,moving_average_price-SELL-54-sec,weighted_moving_average_price-SELL-54-sec,exp_moving_average_price-SELL-54-sec,stochastic_oscillator-SELL-54-sec,trade_volume-SELL-60-sec,stochastic_oscillator-SELL-60-sec
0,0.100,1.000000,1.000000,1.000000,0.0,0.400,1.000078,1.000113,1.000005,0.000000,...,1.000014,1.0,0.000000,3.400,1.000028,1.000018,1.0,1.000000,15.547,0.000000
1,0.200,1.000000,1.000000,1.000000,0.0,0.300,1.000000,1.000000,1.000000,0.000000,...,0.999760,1.0,0.600000,34.697,0.999995,1.000000,1.0,-0.339130,4.384,0.600000
2,0.200,1.000054,1.000093,1.000045,0.0,49.298,0.999927,0.999998,0.999997,0.000000,...,0.999993,1.0,0.000000,22.783,0.999961,0.999998,1.0,0.000000,153.412,0.000000
3,2.513,1.000151,1.000178,1.000007,0.0,4.975,1.000460,1.000613,1.000000,1.000000,...,0.999960,1.0,-0.055696,48.145,0.999830,0.999537,1.0,0.722363,43.943,-0.237069
4,5.152,0.999959,0.999992,0.999995,1.0,0.100,0.999963,0.999992,1.000000,-0.277778,...,1.000007,1.0,-0.111310,19.532,1.000118,1.000278,1.0,0.971310,16.675,-0.111310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38742,6.446,1.000332,1.000367,1.000060,0.5,12.537,1.000000,1.000000,1.000000,0.500000,...,0.999974,1.0,0.000000,45.331,1.000000,1.000000,1.0,0.000000,3.000,0.000000
38743,0.499,1.000000,1.000000,1.000000,0.0,0.000,1.000000,1.000000,1.000000,0.000000,...,0.999932,1.0,0.000000,78.177,0.999982,0.999996,1.0,0.000000,16.877,0.000000
38744,9.999,1.000000,1.000000,1.000000,0.0,0.000,1.000000,1.000000,1.000000,0.000000,...,0.999981,1.0,0.442857,65.138,1.000000,1.000000,1.0,0.119643,604.301,0.442857
38745,58.332,1.000150,1.000011,1.000051,1.0,0.861,1.000411,1.000088,1.000013,0.000000,...,1.000000,1.0,1.000000,88.059,1.000000,1.000000,1.0,0.000000,3.005,1.000000


In [6]:
commision = 0.0005
punch = commision * 4

y = pd.DataFrame(
    df[y_columns].apply(
        lambda row : 1 if max(list(row.to_numpy()), key=abs) > punch else -1 if max(list(row.to_numpy()), key=abs) < -punch else 0,
        axis = 1
    )
)
y = y.rename({0:'target'}, axis='columns')
print(y)
print(y.value_counts())

       target
0           0
1           0
2           1
3           0
4           0
...       ...
38742       0
38743       0
38744       0
38745       0
38746       0

[38747 rows x 1 columns]
target
 0        36607
-1         1155
 1          985
dtype: int64


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle = True, random_state=12)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, train_size=0.75, shuffle = True, random_state=6)

In [8]:
def hyperopt_objective(params):
    model = CatBoostClassifier(
        l2_leaf_reg=int(params['l2_leaf_reg']),
        learning_rate=params['learning_rate'],
        iterations=300,
        eval_metric=metrics.Accuracy(),
        random_seed=126,
        verbose=False,
        loss_function=metrics.MultiClass(),
    )
    
    cv_data = cv(
        Pool(X, y),
        model.get_params(),
        fold_count=4,
        shuffle=True,
        logging_level='Silent',
        stratified=True,
    )

    best_accuracy = np.max(cv_data['test-Accuracy-mean'])
    
    return 1 - best_accuracy


params_space = {
    'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 0, 2, 1),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 5e-1),
}

trials = hyperopt.Trials()

best = hyperopt.fmin(
    hyperopt_objective,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=1,
    trials=trials,
    rstate=RandomState(126)
)

print(best)

  0%|          | 0/3 [00:37<?, ?trial/s, best loss=?]


KeyboardInterrupt: 

In [11]:
params = {
    'iterations': 400,
    #'l2_leaf_reg': int(best['l2_leaf_reg']),
    #'learning_rate': best['learning_rate'],
    'l2_leaf_reg': 2,
    'learning_rate': 0.1,
    'custom_loss': [metrics.Accuracy()],
    'eval_metric': metrics.Accuracy(),
    'random_seed': 42,
    'logging_level': 'Silent',
    'loss_function': 'MultiClass',
}
train_pool = Pool(X_train, y_train)
validate_pool = Pool(X_validation, y_validation)

model = CatBoostClassifier(**params)
model.fit(
    train_pool,
    eval_set=validate_pool,
    plot=True,
    verbose=False,
)

print('Model validation accuracy: {:.4}'.format(
    accuracy_score(y_test, model.predict(X_test))
))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Model validation accuracy: 0.9472


In [12]:
predictions = model.predict(X_test)
predictions = predictions.reshape(predictions.shape[0], 1)
predictions_probs = model.predict_proba(X_test)
print(predictions[90:100])
print(predictions_probs[90:100])
print(y_test[90:100])

[[-1]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]]
[[0.5081676  0.31509127 0.17674113]
 [0.04424105 0.92653006 0.02922889]
 [0.02077968 0.95800285 0.02121747]
 [0.01151404 0.98037193 0.00811403]
 [0.04386998 0.92501239 0.03111763]
 [0.01515275 0.97186993 0.01297732]
 [0.16594606 0.78247401 0.05157993]
 [0.0132275  0.97790602 0.00886648]
 [0.00823896 0.98429562 0.00746542]
 [0.04204708 0.9122708  0.04568212]]
       target
20233      -1
18782       0
26657       0
27456       0
36872       0
25891       0
33455      -1
23349       0
4176        0
37949       0


In [13]:
feature_importances = model.get_feature_importance(train_pool)
feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))


moving_average_price-BUY-12-sec: 2.8777137793896888
moving_average_price-SELL-6-sec: 2.864104855304124
trade_volume-SELL-54-sec: 2.4193466638802734
moving_average_price-BUY-30-sec: 2.385496685878198
trade_volume-BUY-6-sec: 2.2909560754459948
trade_volume-SELL-60-sec: 2.2704596312261947
moving_average_price-SELL-30-sec: 2.2433265405729323
trade_volume-BUY-54-sec: 2.1915861097111873
moving_average_price-BUY-36-sec: 2.1199392333155775
moving_average_price-BUY-48-sec: 2.0380347112355888
trade_volume-SELL-18-sec: 1.9487347734347278
weighted_moving_average_price-BUY-42-sec: 1.9375357255210401
trade_volume-BUY-48-sec: 1.9216786155019963
moving_average_price-BUY-24-sec: 1.789248950222028
moving_average_price-BUY-6-sec: 1.785167211204954
weighted_moving_average_price-BUY-24-sec: 1.7058785165238772
trade_volume-SELL-6-sec: 1.6128284311548704
moving_average_price-SELL-18-sec: 1.59022438928143
moving_average_price-SELL-54-sec: 1.576242904775875
weighted_moving_average_price-BUY-36-sec: 1.565534023

In [15]:
results = []
def f(prob_threshold, diff_threshold, null_threshold):
    p = np.array((list(map(lambda row:
                          1
                          if row[2] > prob_threshold
                              # and row[2] == max(row)
                              and row[2] > row[0]
                              and row[1] < null_threshold
                              and abs(row[2] - row[0]) > diff_threshold
                              and abs(row[2] - row[1]) > diff_threshold
                          else
                          -1
                          if row[0] > prob_threshold
                              # and row[0] == max(row)
                              and row[0] > row[2]
                              and row[1] < null_threshold
                              and abs(row[0] - row[2]) > diff_threshold
                              and abs(row[0] - row[1]) > diff_threshold
                          else 0,
                          predictions_probs))))
    result = {
        'correct_predictions': 0,
        'false_positive': 0,
        'false_negative': 0,
        'wrong_side': 0,
    }
    for i in range(len(p)):
        t = int(y_test['target'][i:i+1])
        if p[i] == t != 0:
            result['correct_predictions'] += 1
        elif p[i] != t and t != 0 and p[i] != 0:
            result['wrong_side'] += 1
        elif p[i] != 0 and t == 0:
            result['false_positive'] += 1
        elif p[i] == 0 and t != 0:
            result['false_negative'] += 1

    result['correct_predictions'] /= len(y_test)
    result['false_positive'] /= len(y_test)
    result['false_negative'] /= len(y_test)
    result['wrong_side'] /= len(y_test)
    result['prob_threshold'] = prob_threshold
    result['diff_threshold'] = diff_threshold
    result['null_threshold'] = null_threshold
    results.append(result)
    return p

for null_threshold in np.arange(0.5, 0.9, 0.1):
    for prob_threshold in np.arange(0.1, 0.7, 0.1):
        for diff_threshold in np.arange(0.00, 0.20, 0.05):
            f(prob_threshold, diff_threshold, null_threshold)

In [29]:
results = list(sorted(results, key=lambda result: -calculate_profit(result['prob_threshold'], result['diff_threshold'], result['null_threshold'])))

In [32]:
print(results[0])

{'correct_predictions': 0.0005161290322580645, 'false_positive': 0.00025806451612903227, 'false_negative': 0.05174193548387097, 'wrong_side': 0.00025806451612903227, 'prob_threshold': 0.6, 'diff_threshold': 0.0, 'null_threshold': 0.5}


In [33]:
prob_threshold = results[0]['prob_threshold']
diff_threshold = results[0]['diff_threshold']
null_threshold = results[0]['null_threshold']

print(f"prob_threshold {prob_threshold}, diff_threshold {diff_threshold}, null_threshold {null_threshold}")
p = f(prob_threshold, diff_threshold, null_threshold)
print('accuracy', accuracy_score(p, y_test))
result = {
    'correct_predictions': 0,
    'false_positive': 0,
    'false_negative': 0,
    'wrong_side': 0,
}
for i in range(len(p)):
    t = int(y_test['target'][i:i+1])
    if p[i] == t != 0:
        result['correct_predictions'] += 1
    elif p[i] != t and t != 0 and p[i] != 0:
        result['wrong_side'] += 1
    elif p[i] != 0 and t == 0:
        result['false_positive'] += 1
    elif p[i] == 0 and t != 0:
        result['false_negative'] += 1

result['correct_predictions_pc'] = result['correct_predictions'] / len(y_test)
result['false_positive_pc'] = result['false_positive'] / len(y_test)
result['false_negative_pc'] = result['false_negative'] / len(y_test)
result['wrong_side_pc'] = result['wrong_side'] / len(y_test)
print(result)

prob_threshold 0.6, diff_threshold 0.0, null_threshold 0.5
accuracy 0.947741935483871
{'correct_predictions': 4, 'false_positive': 2, 'false_negative': 401, 'wrong_side': 2, 'correct_predictions_pc': 0.0005161290322580645, 'false_positive_pc': 0.00025806451612903227, 'false_negative_pc': 0.05174193548387097, 'wrong_side_pc': 0.00025806451612903227}


In [34]:
def calculate_profit(prob_threshold, diff_threshold, null_threshold):
    p = f(prob_threshold, diff_threshold, null_threshold)
    result = {
        'correct_predictions': 0,
        'false_positive': 0,
        'false_negative': 0,
        'wrong_side': 0,
    }
    for i in range(len(p)):
        t = int(y_test['target'][i:i+1])
        if p[i] == t != 0:
            result['correct_predictions'] += 1
        elif p[i] != t and t != 0 and p[i] != 0:
            result['wrong_side'] += 1
        elif p[i] != 0 and t == 0:
            result['false_positive'] += 1
        elif p[i] == 0 and t != 0:
            result['false_negative'] += 1

    result['correct_predictions_pc'] = result['correct_predictions'] / len(y_test)
    result['false_positive_pc'] = result['false_positive'] / len(y_test)
    result['false_negative_pc'] = result['false_negative'] / len(y_test)
    result['wrong_side_pc'] = result['wrong_side'] / len(y_test)
    
    return (
          len(y) * result['correct_predictions_pc'] * (punch - 2 * commision)
          -
          len(y) * result['false_positive_pc'] * 2 * commision
          -
          len(y) * result['wrong_side_pc'] * (punch + 2 * commision)
    )
profit = calculate_profit(prob_threshold, diff_threshold, null_threshold)
print(
    'profit from 3000$ fot 2 months',
    profit * 3000,
    '$',
    profit * 100,
    '%'
)

profit from 3000$ fot 2 months -59.99535483870969 $ -1.999845161290323 %


In [35]:
correct_accuracy = result['correct_predictions'] / len(y_test[y_test['target'] != 0])
print(
f"Correct accuracy = correct prediction / total positive num = {correct_accuracy}"
)
FPR = result['false_positive'] / len(y_test[y_test['target'] == 0])
print(
f"FPR = false positive / total negative num = {FPR}"
)
FNR = result['false_negative'] / len(y_test[y_test['target'] != 0])
print(
f"FNR = false negative / total positive num = {FNR}"
)
cases_pc = (result['correct_predictions'] + result['false_positive'] + result['wrong_side']) / len(y_test[y_test['target'] != 0]) * 100
probability_pc = result['correct_predictions'] / (result['correct_predictions'] + result['false_positive'] + result['wrong_side']) * 100
print(
f"Model in {cases_pc}% of cases with probability {probability_pc}% predict a correct market jump"
)

Correct accuracy = correct prediction / total positive num = 0.009828009828009828
FPR = false positive / total negative num = 0.00027236824186299875
FNR = false negative / total positive num = 0.9852579852579852
Model in 1.9656019656019657% of cases with probability 50.0% predict a correct market jump


In [22]:
for i in range(len(predictions_probs)):
    if int(y_test['target'][i:i+1]) != 0:
        print(p[i], predictions_probs[i], int(y_test['target'][i:i+1]))

0 [0.01471173 0.97006906 0.01521921] 1
0 [0.0348011  0.90680791 0.05839099] -1
0 [0.00963603 0.97766475 0.01269922] -1
0 [0.01653236 0.95028703 0.03318061] -1
0 [0.52221957 0.09492226 0.38285818] 1
0 [0.07922507 0.85985502 0.06091991] -1
0 [0.04815805 0.89459932 0.05724264] -1
0 [0.01195401 0.97746588 0.01058011] 1
0 [0.12395765 0.80260804 0.07343431] -1
0 [0.5081676  0.31509127 0.17674113] -1
0 [0.16594606 0.78247401 0.05157993] -1
0 [0.04370489 0.90159804 0.05469707] 1
0 [0.07440299 0.89156261 0.0340344 ] -1
0 [0.21968114 0.57978941 0.20052945] 1
0 [0.04477235 0.93471579 0.02051185] 1
0 [0.15696873 0.56522061 0.27781066] -1
0 [0.15766214 0.63325357 0.2090843 ] 1
0 [0.03348667 0.95488829 0.01162503] -1
0 [0.02321405 0.95758024 0.0192057 ] 1
0 [0.07689473 0.87340264 0.04970263] 1
0 [0.01855031 0.95707859 0.0243711 ] 1
0 [0.0123305  0.97988634 0.00778316] 1
0 [0.29304932 0.55929483 0.14765585] -1
0 [0.01446145 0.97584037 0.00969818] -1
0 [0.49208424 0.12162241 0.38629335] 1
0 [0.0933631